In [1]:
!pip install pyspark==3.5.5

In [2]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import os
ROOT_DIR = '/workspace/NN'
os.chdir(ROOT_DIR)

import shutil
import kagglehub
import torch
from pyspark.sql import SparkSession
import socket

dataset_path = os.path.join(ROOT_DIR, 'neural', 'datasets', 'spark', 'test_1')
weight_path = os.path.join(ROOT_DIR, 'neural', 'weights', 'lab_5', 'kmeans')
os.makedirs(dataset_path, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



print(socket.gethostbyname("spark-master"))
driver_host = socket.gethostbyname(socket.gethostname())
print(driver_host)
driver_host = socket.gethostbyname("spark-master")
print(driver_host)
driver_host = "producer"
os.environ["PYSPARK_SUBMIT_ARGS"] = f"""
--master spark://spark-master:7077
--conf spark.driver.host={driver_host}
--conf spark.driver.port=45555
--conf spark.executor.memory=1G
--conf spark.executor.cores=1
pyspark-shell
"""
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'  # путь к Python в контейнере
# os.environ['SPARK_HOME'] = '/opt/spark'            # путь к Spark, укажи реальный
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'

MONGO_USER = os.environ["MONGO_USER"]
MONGO_PASS = os.environ["MONGO_PASSWORD"]
MONGO_ADDR = f"{MONGO_USER}:{MONGO_PASS}@mongodb:27017"  # :27017

def spark_app_generator(name):
    spark = SparkSession.builder \
        .master("spark://spark-master:7077") \
        .appName(name) \
         .config("spark.driver.bindAddress", "0.0.0.0") \
        .config("spark.executor.memory", "1g") \
        .config("spark.driver.memory", "1g") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.3.0") \
    .config("spark.mongodb.read.connection.uri", f"mongodb://{MONGO_ADDR}") \
    .config("spark.mongodb.write.connection.uri", f"mongodb://{MONGO_ADDR}") \
        .getOrCreate()

        # .config("spark.hadoop.hadoop.home.dir", "/hadoop/dfs/name") \
    # .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    # .config("spark.hadoop.dfs.client.use.namenode.hostname", "true") \
    return spark  # /openfoodfacts.products


172.20.0.5
172.21.0.2
172.20.0.5


In [3]:

# spark.stop()
spark = spark_app_generator('test_mongo_reading')
# Проверка: записать и прочитать файл
# Создать директорию в HDFS
# spark._jsc.hadoopConfiguration().set("dfs.client.block.write.replace-datanode-on-failure.enable", "true")
# spark._jsc.hadoopConfiguration().set("dfs.client.use.namenode.hostname", "true")

# Запись простого DataFrame
df = spark.createDataFrame([("hello",),("world",), ('fdf',)], ["value45",])
# df.write.mode("overwrite").text("hdfs://namenode:9000/tmp/hadoop-root/dfs/name")
df.write.mode("overwrite").text("/opt/bitnami/spark/work/labs/lab_5/some.txt")

# Чтение назад

df2 = spark.read.text("/opt/bitnami/spark/work/labs/lab_5/some.txt")
df2.show()

# spark.stop()
# df = spark.read.text("hdfs://datanode:9000/user/root/input/sample.txt")
# df.show()

# spark.stop()

:: loading settings :: url = jar:file:/usr/local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e8109d23-93c9-4836-bfc3-ee9d18fd9ccd;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.3.0 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
downloading https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/10.3.0/mongo-spark-connector_2.12-10.3.0.jar ...
	[SUCCESSFUL ] org.mongodb.spark#mongo-spark-connector_2.12;10.3.0!mongo-spark-connector_2.12.jar (222ms)
downloading https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-sync/4.8.2/mongodb-driver-sync-4.8.2.jar ...
	[SUCCESSFUL ] org.

+-----+
|value|
+-----+
+-----+



In [4]:
from pyspark.sql.types import (StructType, StructField,
                               StringType, IntegerType, MapType,
                               DoubleType
                               )

# Пример определения схемы. Настройте схему под структуру ваших данных.
custom_schema = StructType([
    StructField("_id", StringType(), True),
    StructField("product_name", StringType(), True),
    # Если nutriments представляет собой динамические поля, лучше сохранить его как MapType.
    StructField("nutriments", MapType(StringType(), StringType()), True),
    # Если есть другие поля, укажите их типы.
    StructField("quantity", StringType(), True),
     StructField("ingredients_sweeteners_n", IntegerType(), True),
     StructField("ingredients_percent_analysis", IntegerType(), True),
     StructField("ingredients_non_nutritive_sweeteners_n", IntegerType(), True),
     StructField("ingredients_n", IntegerType(), True),
    StructField("ingredients_from_palm_oil_n", IntegerType(), True),
    StructField("ingredients_from_or_that_may_be_from_palm_oil_n", IntegerType(), True),
    StructField("additives_n", IntegerType(), True),
    StructField("unique_scans_n", IntegerType(), True),
    StructField("scans_n", IntegerType(), True),
    StructField("rev", IntegerType(), True),
    StructField("popularity_key", IntegerType(), True),
    StructField("packagings_n", IntegerType(), True),
    StructField("packagings_complete", IntegerType(), True),
    StructField("nutrition_score_warning_no_fiber", IntegerType(), True),
    StructField("nutrition_score_warning_fruits_vegetables_nuts_estimate", IntegerType(), True),
    StructField("nutrition_score_beverage", IntegerType(), True),
    StructField("nutriscore_score_opposite", IntegerType(), True),
    StructField("nutriscore_score", IntegerType(), True),
    StructField("environmental_score_score", IntegerType(), True),
    StructField("completeness", DoubleType(), True),
    StructField("complete", IntegerType(), True),

])

spark = spark_app_generator('test_mongo_reading')
df = spark.read.schema(custom_schema).format("mongodb") \
  .options(host="mongo:27017", database="off", collection='products').load()  # , database="off", collection='products'

# Просмотр схемы и первых строк
df.printSchema()
df.show(20)


root
 |-- _id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- nutriments: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- quantity: string (nullable = true)
 |-- ingredients_sweeteners_n: integer (nullable = true)
 |-- ingredients_percent_analysis: integer (nullable = true)
 |-- ingredients_non_nutritive_sweeteners_n: integer (nullable = true)
 |-- ingredients_n: integer (nullable = true)
 |-- ingredients_from_palm_oil_n: integer (nullable = true)
 |-- ingredients_from_or_that_may_be_from_palm_oil_n: integer (nullable = true)
 |-- additives_n: integer (nullable = true)
 |-- unique_scans_n: integer (nullable = true)
 |-- scans_n: integer (nullable = true)
 |-- rev: integer (nullable = true)
 |-- popularity_key: integer (nullable = true)
 |-- packagings_n: integer (nullable = true)
 |-- packagings_complete: integer (nullable = true)
 |-- nutrition_score_warning_no_fiber: integer (nullable = true)
 |-- nutritio

+------------+--------------------+--------------------+------------------+------------------------+----------------------------+--------------------------------------+-------------+---------------------------+-----------------------------------------------+-----------+--------------+-------+---+--------------+------------+-------------------+--------------------------------+-------------------------------------------------------+------------------------+-------------------------+----------------+-------------------------+------------------+--------+
|         _id|        product_name|          nutriments|          quantity|ingredients_sweeteners_n|ingredients_percent_analysis|ingredients_non_nutritive_sweeteners_n|ingredients_n|ingredients_from_palm_oil_n|ingredients_from_or_that_may_be_from_palm_oil_n|additives_n|unique_scans_n|scans_n|rev|popularity_key|packagings_n|packagings_complete|nutrition_score_warning_no_fiber|nutrition_score_warning_fruits_vegetables_nuts_estimate|nutrition

In [5]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

spark = spark_app_generator('test_mongo_data_analysis')
df = spark.read.schema(custom_schema).format("mongodb") \
  .options(host="mongo:27017", database="off", collection='products').load()

# Выбор числовых признаков
numeric_features = [ 'popularity_key', 'popularity_key', "rev"]

# Удаление строк с пропущенными значениями
df_clean = df.select(numeric_features).dropna()

# Сбор признаков в вектор
assembler = VectorAssembler(inputCols=numeric_features, outputCol="features_assembled")
df_vector = assembler.transform(df_clean)

# Масштабирование признаков
scaler = StandardScaler(inputCol="features_assembled", outputCol="features")
scaler_model = scaler.fit(df_vector)
df_scaled = scaler_model.transform(df_vector)

25/04/22 15:53:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator


# df_small = df_scaled.limit(100)
# Обучение модели k-средних
kmeans = KMeans().setK(4).setSeed(1).setFeaturesCol("features")
model = kmeans.fit(df_scaled)



25/04/22 16:03:58 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [7]:
# Предсказание кластеров
predictions = model.transform(df_scaled)

# Оценка модели
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette Score: {silhouette}")

# Центры кластеров
centers = model.clusterCenters()
for idx, center in enumerate(centers):
    print(f"Cluster {idx} center: {center}")

Silhouette Score: 0.9747334431984966
Cluster 0 center: [4.09751838 4.09751838 1.73215213]
Cluster 1 center: [0.00162783 0.00162783 0.47424602]
Cluster 2 center: [-4.76211024 -4.76211024  2.14917652]
Cluster 3 center: [  0.43211761   0.43211761 353.37543358]


In [8]:

# weight_path = os.path.join(ROOT_DIR, 'neural', 'weights', 'lab_5', 'kmeans_3')
# os.makedirs(weight_path, exist_ok=True)
# model.save("/opt/bitnami/spark/work/labs/lab_5/kmeans_5")
model.save('/opt/bitnami/spark/work/labs/lab_5/kmeans_6')

In [4]:
# Не забудьте остановить SparkSession по завершении работы
spark.stop()

In [10]:
from pyspark.ml.clustering import KMeansModel

loaded_model = KMeansModel.load("/opt/bitnami/spark/work/labs/lab_5/kmeans_6")


UnsupportedOperationException: empty collection

In [12]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("KMeansExample") \
    .getOrCreate()

# Пример данных
data = [
    (0.0, 0.0),
    (1.0, 1.0),
    (9.0, 8.0),
    (8.0, 9.0)
]
columns = ["x", "y"]
df = spark.createDataFrame(data, columns)

# Преобразование признаков в вектор
assembler = VectorAssembler(inputCols=["x", "y"], outputCol="features")
dataset = assembler.transform(df)

# Обучение модели KMeans
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Сохранение модели
model.save("/opt/bitnami/spark/work/labs/lab_5/kmeans_test")

# Загрузка модели
from pyspark.ml.clustering import KMeansModel
loaded_model = KMeansModel.load("/opt/bitnami/spark/work/labs/lab_5/kmeans_test")

# Применение загруженной модели
predictions = loaded_model.transform(dataset)
predictions.show()


25/04/22 14:25:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


UnsupportedOperationException: empty collection